# Collaboration based recommender system

In [ ]:
 !pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357256 sha256=a406f36664b14cc7bb02c0c63e1842d7c770eef80ecfeed42fa4864d0dbb540f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from ast import literal_eval
books_df = pd.read_csv('data/books_enriched.csv',  index_col=[0], converters={"genres": literal_eval})
ratings = pd.read_csv('data/ratings.csv', converters={"genres": literal_eval})

In [ ]:

book_cols= ['book_id', 'title','description']
books_df = books_df.dropna(subset=['description'])
books_df = books_df[book_cols]
book_rating = ratings.merge(books_df, on='book_id')


In [ ]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [ ]:

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(book_rating[['user_id', 'book_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)


algo = SVD()
algo.fit(trainset)

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
class CollaborativeRecommender:
    def __init__(self, algo, trainset, testset):
        self.algo = algo
        self.trainset = trainset
        self.testset = testset

    def predict(self, user_id, k=10):
        try:
            # Convert raw user_id to inner user_id
            inner_user_id = self.trainset.to_inner_uid(user_id)
        except ValueError:
            # User not found in trainset, return None
            return None

        # Get the list of books the user has interacted with
        user_books = set(self.trainset.ur[inner_user_id])
        all_books = set(self.trainset.all_items())
        unseen_books = all_books - user_books
        # Predict the ratings for unseen books
        predictions = [self.algo.predict(self.trainset.to_raw_uid(inner_user_id), self.trainset.to_raw_iid(book_id)) for book_id in unseen_books]
        # Sort the predictions by estimated rating and return the top-k books
        top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:k]
        top_books = [pred.iid for pred in top_predictions]
        return top_books
    def evaluate(self, k=10):
        predictions = self.algo.test(self.testset)
        hit_rate, ndcg_scores = [], []
        count=0
        for user_id in set(pred.uid for pred in predictions):
            try:
                inner_user_id = self.trainset.to_inner_uid(user_id)
            except ValueError:
                pred_books = None
            else:
                pred_books = set(self.predict(user_id, k))

            true_books = set(book_id for (uid, book_id, _) in self.testset if uid == user_id)
            #print("true book {}".format(len(true_books)))
            if pred_books is not None:
                hits = len(true_books & pred_books)
                denom = min(k, len(true_books))
                if denom > 0:
                    hit_rate.append(hits / denom)

                dcg = sum(1 / np.log2(rank + 2) for rank, book in enumerate(pred_books) if book in true_books)
                idcg = sum(1 / np.log2(i + 2) for i in range(min(k, len(true_books))))
                ndcg = dcg / idcg if idcg > 0 else 0
                ndcg_scores.append(ndcg)

            count+=1
            if count==100:
              break

        return np.mean(hit_rate), np.mean(ndcg_scores)

dùng map@12 làm metrics

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
class CollaborativeRecommender:
    def __init__(self, algo, trainset, testset):
        self.algo = algo
        self.trainset = trainset
        self.testset = testset

    def predict(self, user_id, k=10):
        try:
            # Convert raw user_id to inner user_id
            inner_user_id = self.trainset.to_inner_uid(user_id)
        except ValueError:
            # User not found in trainset, return None
            return None

        # Get the list of books the user has interacted with
        user_books = set(self.trainset.ur[inner_user_id])
        all_books = set(self.trainset.all_items())
        unseen_books = all_books - user_books
        # Predict the ratings for unseen books
        predictions = [self.algo.predict(self.trainset.to_raw_uid(inner_user_id), self.trainset.to_raw_iid(book_id)) for book_id in unseen_books]
        # Sort the predictions by estimated rating and return the top-k books
        top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:k]
        top_books = [pred.iid for pred in top_predictions]
        return top_books
    def evaluate(self, k=12):  # Evaluate at k=12 for MAP@12
        predictions = self.algo.test(self.testset)
        average_precisions = []

        for user_id in set(pred.uid for pred in predictions):
            try:
                inner_user_id = self.trainset.to_inner_uid(user_id)
            except ValueError:
                continue  # Skip users not in the training set

            pred_books = self.predict(user_id, k)  # Get top-k predictions
            true_books = set(book_id for (uid, book_id, _) in self.testset if uid == user_id)

            # Create binary relevance labels for each prediction (1 if relevant, 0 if not)
            relevance_labels = [1 if book in true_books else 0 for book in pred_books]

            # If there are no relevant books, skip the calculation
            if sum(relevance_labels) > 0:
                average_precision = average_precision_score(relevance_labels, [pred.est for pred in predictions if pred.uid == user_id][:k])
                average_precisions.append(average_precision)

        # Return the mean of the average precision scores
        return np.mean(average_precisions)

In [ ]:
collab_recommender = CollaborativeRecommender(algo, trainset, testset)



In [ ]:
import time

start_time = time.time()
hit_rate, ndcg = collab_recommender.evaluate(k=10)
print(f"Hit Rate: {hit_rate:.4f}")
print(f"NDCG: {ndcg:.4f}")
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time:.4f} seconds")

Hit Rate: 0.0110
NDCG: 0.0117
Elapsed time: 62.6926 seconds


In [ ]:
pip install prometheus-client

In [ ]:
!pip show prometheus-client

Name: prometheus_client
Version: 0.20.0
Summary: Python client for the Prometheus monitoring system.
Home-page: https://github.com/prometheus/client_python
Author: Brian Brazil
Author-email: brian.brazil@robustperception.io
License: Apache Software License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: jupyter-server, notebook


In [ ]:
!pip install python-dotenv